In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support

## Own code 
import import_ipynb
import metadata_options
import models_nn

Using TensorFlow backend.


importing Jupyter notebook from metadata_options.ipynb
importing Jupyter notebook from models_nn.ipynb


In [2]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [3]:
######################################
## Weighted Onehot Encoding options ##
######################################

##############
# Throughput #
##############
# TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP3 + k: weighted by 1 inverted k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP4 + k: weighted by 1 upright k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# (For TP3 & TP4, k=1 results in V-shaped variance, and as k>1 increases, sides will curve into U-shaped variance)

############
# Worktime #
############
# WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# WT2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

################
# PC agreement #
################
# PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)

#####################
# Input text length #
#####################
# TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

###################
# Special Options #
###################
# SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP2: weighted by average of WT1 and WT2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP3: weighted by average of PC1 and PC2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_UNI: weighted by 1 uniformly distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_NORM: weighted by 1 normally distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

# Select 1 option from each of the few variants above, e.g. TP2, WT1, PC3, TL1, SP3, and input into function
# set_OHE_pipeline_options. If not selecting TP3 or TP4, input k (option_k) will be ignored. After
# editing the options, run the entire notebook for results accordingly.

# Edit option choices here
throughput_option = 'TP1'
worktime_option = 'WT1'
pc_agreement_option = 'PC1'
textlength_option = 'TL1'
special_option = 'SP1'
k_option_for_tp = 3

df_throughput, df_worktime, df_agreement, df_textlength, df_special = metadata_options.set_OHE_pipeline_options(df, throughput_option, worktime_option, pc_agreement_option, textlength_option, special_option, k_option_for_tp)

TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)


In [4]:
######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn

model_name = 'lstm'
models_nn.MODEL_NAME = model_name

In [5]:
df.head()

,Input.sentence_id,HITId,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,Input.speaker_intention,...,prt,punct,purpcl,quantmod,rcmod,rel,root,tmod,xcomp,xsubj
5,11,3MG8450X2OASXZ0WO9O5AH70GU3UPA,Game1-italy-germany,Train,Game1-italy-germany-3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,Truth,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,12,38G0E1M85M552JXSALX4G9WI2I6UVX,Game1-italy-germany,Train,Game1-italy-germany-7,117,"Sorry Italy I've been away doing, um, German t...",germany-Game1,Game1-italy-germany-6,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,14,3HYV4299H0WQ2B4TCS7PKDQ75WHE81,Game1-italy-germany,Train,Game1-italy-germany-8,119,"I don't think I'm ready to go for that idea, h...",germany-Game1,Game1-italy-germany-7,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,15,3XU9MCX6VOC4P079IHIO9TCNYLGR2P,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am pretty conflicted about whether to guess ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,16,3FVBZG9CLJEK4WQS7P2GC1H2EEQH0Q,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am going to take it literally and say even ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0


In [6]:
# # Old train_test_split code
# train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

# New train_test_split using Stratified Shaffled Splits
y = df["Input.deception_quadrant"].copy()
X = df.drop(["Input.deception_quadrant"], axis=1)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [7]:
print(df.shape, train.shape, test.shape)

(11366, 862) (9092, 862) (2274, 862)


In [8]:
# check scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.23.2


In [9]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)
deception_class_weight_dict = dict(enumerate(deception_class_weights))

y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
rapport_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_rapport),
                                                 y_train_rapport)
rapport_class_weight_dict = dict(enumerate(rapport_class_weights))

y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
share_info_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_share_information),
                                                 y_train_share_information)
share_info_class_weight_dict = dict(enumerate(share_info_class_weights))

y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
reasoning_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_reasoning),
                                                 y_train_reasoning)
reasoning_class_weight_dict = dict(enumerate(reasoning_class_weights))

y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()
gamemove_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_gamemove),
                                                 y_train_gamemove)
gamemove_class_weight_dict = dict(enumerate(gamemove_class_weights))

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [10]:
print("Class weights")
print("Deception: {} \nRapport: {} \nShare Information: {} \nReasoning: {} \nGamemove: {}".format(deception_class_weight_dict,
                                                                                                  rapport_class_weight_dict,
                                                                                                  share_info_class_weight_dict,
                                                                                                  reasoning_class_weight_dict,
                                                                                                  gamemove_class_weight_dict))

Class weights
Deception: {0: 10.192825112107624, 1: 0.525792273883877} 
Rapport: {0: 3.582348305752561, 1: 0.5811069922024799} 
Share Information: {0: 3.117969821673525, 1: 0.59549384333246} 
Reasoning: {0: 2.9596354166666665, 1: 0.6016410799364743} 
Gamemove: {0: 7.048062015503876, 1: 0.5381792352314431}


In [11]:
X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [12]:
le = LabelEncoder()

# def label_preprocessing(y_data, label_encoder):
#     out = label_encoder.fit_transform(y_data).reshape(-1,1)
#     return out

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [13]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

### Construct individual  models

In [14]:
# Rapport model
rapport_model = models_nn.create_nn_model()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
rapport_model.fit(X_train,y_train_rapport,
                  batch_size=128,
                  epochs=50,
                  validation_data=(X_test, y_test_rapport), 
#                   callbacks=[models_nn.early_stop],
                  class_weight=rapport_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 45ms/step - loss: 0.6936 - accuracy: 0.5505 - f1_m: 0.5975 - recall_m: 0.5660 - precision_m: 0.8532 - val_loss: 0.7054 - val_accuracy: 0.1341 - val_f1_m: 0.0146 - val_recall_m: 0.0075 - val_precision_m: 0.3222
Epoch 2/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6887 - accuracy: 0.4879 - f1_m: 0.5376 - recall_m: 0.4562 - precision_m: 0.8760 - val_loss: 0.7388 - val_accuracy: 0.4099 - val_f1_m: 0.5218 - val_recall_m: 0.3722 - val_precision_m: 0.8829
Epoch 3/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6760 - accuracy: 0.6094 - f1_m: 0.7212 - recall_m: 0.6204 - precision_m: 0.8971 - val_loss: 0.6095 - val_accuracy: 0.7318 - val_f1_m: 0.8398 - val_recall_m: 0.8143 - val_precision_m: 0.8679
Epoch 4/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6527 - accuracy: 0.6273 - f1_m: 0.7384 - recall_m: 0.6320 - precision_m: 0.9125 - val_loss: 0.5378 - val_accuracy: 0.7687 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3187 - accuracy: 0.7959 - f1_m: 0.8622 - recall_m: 0.7694 - precision_m: 0.9880 - val_loss: 0.9910 - val_accuracy: 0.5532 - val_f1_m: 0.6909 - val_recall_m: 0.5763 - val_precision_m: 0.8647
Epoch 33/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3102 - accuracy: 0.8008 - f1_m: 0.8678 - recall_m: 0.7764 - precision_m: 0.9887 - val_loss: 1.3440 - val_accuracy: 0.6029 - val_f1_m: 0.7367 - val_recall_m: 0.6429 - val_precision_m: 0.8646
Epoch 34/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3062 - accuracy: 0.8075 - f1_m: 0.8758 - recall_m: 0.7889 - precision_m: 0.9884 - val_loss: 1.3763 - val_accuracy: 0.5998 - val_f1_m: 0.7329 - val_recall_m: 0.6355 - val_precision_m: 0.8671
Epoch 35/50
72/72 [==============================] - 2s 35ms/step - loss: 0.3007 - accuracy: 0.8068 - f1_m: 0.8745 - recall_m: 0.7882 - precision_m: 0.9887 - val_loss: 1.4657 - val_accuracy: 0.6957 - va

In [15]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

(0.4900503181226442, 0.48036724307542816, 0.45867371062479456, None)

In [16]:
# Game move model
gamemove_model = models_nn.create_nn_model()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                               models_nn.recall_m, models_nn.precision_m])
gamemove_model.fit(X_train,
                   y_train_gamemove,
                   batch_size=128,
                   epochs=50,
                   validation_data=(X_test, y_test_gamemove), 
#                    callbacks=[models_nn.early_stop],
                   class_weight=gamemove_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 41ms/step - loss: 0.6935 - accuracy: 0.7685 - f1_m: 0.8429 - recall_m: 0.8126 - precision_m: 0.9291 - val_loss: 0.6928 - val_accuracy: 0.5079 - val_f1_m: 0.6549 - val_recall_m: 0.5012 - val_precision_m: 0.9505
Epoch 2/50
72/72 [==============================] - 2s 35ms/step - loss: 0.6818 - accuracy: 0.6288 - f1_m: 0.7483 - recall_m: 0.6417 - precision_m: 0.9508 - val_loss: 0.5737 - val_accuracy: 0.8333 - val_f1_m: 0.9075 - val_recall_m: 0.8751 - val_precision_m: 0.9435
Epoch 3/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6428 - accuracy: 0.6688 - f1_m: 0.7767 - recall_m: 0.6699 - precision_m: 0.9581 - val_loss: 0.6085 - val_accuracy: 0.6425 - val_f1_m: 0.7752 - val_recall_m: 0.6596 - val_precision_m: 0.9432
Epoch 4/50
72/72 [==============================] - 2s 34ms/step - loss: 0.5960 - accuracy: 0.6468 - f1_m: 0.7661 - recall_m: 0.6428 - precision_m: 0.9686 - val_loss: 1.0962 - val_accuracy: 0.2520 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 34ms/step - loss: 0.2809 - accuracy: 0.8384 - f1_m: 0.9048 - recall_m: 0.8334 - precision_m: 0.9947 - val_loss: 0.8583 - val_accuracy: 0.7265 - val_f1_m: 0.8391 - val_recall_m: 0.7618 - val_precision_m: 0.9349
Epoch 33/50
72/72 [==============================] - 2s 34ms/step - loss: 0.2652 - accuracy: 0.8492 - f1_m: 0.9098 - recall_m: 0.8442 - precision_m: 0.9921 - val_loss: 1.2411 - val_accuracy: 0.4868 - val_f1_m: 0.6386 - val_recall_m: 0.4841 - val_precision_m: 0.9423
Epoch 34/50
72/72 [==============================] - 2s 34ms/step - loss: 0.2647 - accuracy: 0.8350 - f1_m: 0.8998 - recall_m: 0.8249 - precision_m: 0.9964 - val_loss: 0.9555 - val_accuracy: 0.7383 - val_f1_m: 0.8469 - val_recall_m: 0.7718 - val_precision_m: 0.9397
Epoch 35/50
72/72 [==============================] - 2s 34ms/step - loss: 0.2506 - accuracy: 0.8546 - f1_m: 0.9099 - recall_m: 0.8433 - precision_m: 0.9952 - val_loss: 1.0146 - val_accuracy: 0.8140 - va

In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.5013883596875596, 0.5038514388394155, 0.4802507648025076, None)

In [18]:
# Reasoning model
reasoning_model = models_nn.create_nn_model()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
reasoning_model.fit(X_train,
                    y_train_reasoning,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_reasoning), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=reasoning_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 40ms/step - loss: 0.6934 - accuracy: 0.4858 - f1_m: 0.5264 - recall_m: 0.4794 - precision_m: 0.7440 - val_loss: 0.7103 - val_accuracy: 0.2142 - val_f1_m: 0.0937 - val_recall_m: 0.0499 - val_precision_m: 0.8886
Epoch 2/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6878 - accuracy: 0.5577 - f1_m: 0.6496 - recall_m: 0.5560 - precision_m: 0.8642 - val_loss: 0.7138 - val_accuracy: 0.4055 - val_f1_m: 0.4804 - val_recall_m: 0.3371 - val_precision_m: 0.8451
Epoch 3/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6656 - accuracy: 0.5958 - f1_m: 0.7010 - recall_m: 0.5929 - precision_m: 0.8848 - val_loss: 0.7232 - val_accuracy: 0.4666 - val_f1_m: 0.5735 - val_recall_m: 0.4393 - val_precision_m: 0.8314
Epoch 4/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6412 - accuracy: 0.6248 - f1_m: 0.7286 - recall_m: 0.6209 - precision_m: 0.9016 - val_loss: 0.5914 - val_accuracy: 0.6992 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3217 - accuracy: 0.7973 - f1_m: 0.8622 - recall_m: 0.7716 - precision_m: 0.9807 - val_loss: 1.3399 - val_accuracy: 0.6218 - val_f1_m: 0.7465 - val_recall_m: 0.6829 - val_precision_m: 0.8251
Epoch 33/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3151 - accuracy: 0.7978 - f1_m: 0.8593 - recall_m: 0.7697 - precision_m: 0.9794 - val_loss: 1.3709 - val_accuracy: 0.6099 - val_f1_m: 0.7346 - val_recall_m: 0.6611 - val_precision_m: 0.8291
Epoch 34/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3053 - accuracy: 0.8013 - f1_m: 0.8624 - recall_m: 0.7724 - precision_m: 0.9810 - val_loss: 1.3896 - val_accuracy: 0.6601 - val_f1_m: 0.7799 - val_recall_m: 0.7377 - val_precision_m: 0.8291
Epoch 35/50
72/72 [==============================] - 2s 34ms/step - loss: 0.2987 - accuracy: 0.8088 - f1_m: 0.8728 - recall_m: 0.7872 - precision_m: 0.9837 - val_loss: 1.5993 - val_accuracy: 0.6108 - va

In [19]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.5161728952051533, 0.5204585537918871, 0.5145522404271867, None)

In [20]:
# Share Information model
shareinfo_model = models_nn.create_nn_model()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
shareinfo_model.fit(X_train,
                    y_train_share_information,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_share_information), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=share_info_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 40ms/step - loss: 0.6933 - accuracy: 0.5748 - f1_m: 0.6385 - recall_m: 0.6141 - precision_m: 0.8312 - val_loss: 0.6852 - val_accuracy: 0.7744 - val_f1_m: 0.8695 - val_recall_m: 0.8989 - val_precision_m: 0.8432
Epoch 2/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6884 - accuracy: 0.6090 - f1_m: 0.7070 - recall_m: 0.6302 - precision_m: 0.8527 - val_loss: 0.7134 - val_accuracy: 0.4028 - val_f1_m: 0.4901 - val_recall_m: 0.3416 - val_precision_m: 0.8748
Epoch 3/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6656 - accuracy: 0.5919 - f1_m: 0.6984 - recall_m: 0.5887 - precision_m: 0.8938 - val_loss: 0.5378 - val_accuracy: 0.7608 - val_f1_m: 0.8602 - val_recall_m: 0.8804 - val_precision_m: 0.8424
Epoch 4/50
72/72 [==============================] - 2s 34ms/step - loss: 0.6436 - accuracy: 0.6158 - f1_m: 0.7243 - recall_m: 0.6119 - precision_m: 0.9072 - val_loss: 0.6739 - val_accuracy: 0.5633 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 34ms/step - loss: 0.3686 - accuracy: 0.7271 - f1_m: 0.8020 - recall_m: 0.6794 - precision_m: 0.9878 - val_loss: 1.1528 - val_accuracy: 0.5422 - val_f1_m: 0.6705 - val_recall_m: 0.5544 - val_precision_m: 0.8508
Epoch 33/50
72/72 [==============================] - 2s 35ms/step - loss: 0.3700 - accuracy: 0.7211 - f1_m: 0.7967 - recall_m: 0.6735 - precision_m: 0.9859 - val_loss: 1.8285 - val_accuracy: 0.5893 - val_f1_m: 0.7174 - val_recall_m: 0.6213 - val_precision_m: 0.8511
Epoch 34/50
72/72 [==============================] - 3s 35ms/step - loss: 0.3735 - accuracy: 0.7215 - f1_m: 0.7942 - recall_m: 0.6713 - precision_m: 0.9873 - val_loss: 0.9974 - val_accuracy: 0.4982 - val_f1_m: 0.6219 - val_recall_m: 0.4912 - val_precision_m: 0.8505
Epoch 35/50
72/72 [==============================] - 3s 35ms/step - loss: 0.3615 - accuracy: 0.7243 - f1_m: 0.8068 - recall_m: 0.6856 - precision_m: 0.9876 - val_loss: 1.5480 - val_accuracy: 0.6069 - va

In [21]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.5079539028229936, 0.5114848163841808, 0.5017527006533157, None)

In [22]:
# Deception model
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_train,
                    y_train_deception,
                    epochs=50,
                    validation_data=(X_test, y_test_deception), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=deception_class_weight_dict)

Epoch 1/50
285/285 [==============================] - 8s 27ms/step - loss: 0.6952 - accuracy: 0.7696 - f1_m: 0.8411 - recall_m: 0.8022 - precision_m: 0.9527 - val_loss: 0.6379 - val_accuracy: 0.9503 - val_f1_m: 0.9745 - val_recall_m: 0.9996 - val_precision_m: 0.9514
Epoch 2/50
285/285 [==============================] - 7s 25ms/step - loss: 0.6897 - accuracy: 0.6879 - f1_m: 0.7761 - recall_m: 0.7041 - precision_m: 0.9580 - val_loss: 0.6111 - val_accuracy: 0.7449 - val_f1_m: 0.8520 - val_recall_m: 0.7719 - val_precision_m: 0.9560
Epoch 3/50
285/285 [==============================] - 7s 25ms/step - loss: 0.6702 - accuracy: 0.6142 - f1_m: 0.7301 - recall_m: 0.6146 - precision_m: 0.9665 - val_loss: 0.6310 - val_accuracy: 0.6060 - val_f1_m: 0.7479 - val_recall_m: 0.6195 - val_precision_m: 0.9564
Epoch 4/50
285/285 [==============================] - 7s 24ms/step - loss: 0.6624 - accuracy: 0.5930 - f1_m: 0.7180 - recall_m: 0.5914 - precision_m: 0.9727 - val_loss: 0.6701 - val_accuracy: 0.5220 

Epoch 32/50
285/285 [==============================] - 7s 24ms/step - loss: 0.4893 - accuracy: 0.7552 - f1_m: 0.8500 - recall_m: 0.7537 - precision_m: 0.9873 - val_loss: 0.7734 - val_accuracy: 0.5330 - val_f1_m: 0.6818 - val_recall_m: 0.5365 - val_precision_m: 0.9516
Epoch 33/50
285/285 [==============================] - 7s 24ms/step - loss: 0.4556 - accuracy: 0.7651 - f1_m: 0.8554 - recall_m: 0.7624 - precision_m: 0.9885 - val_loss: 0.4911 - val_accuracy: 0.7493 - val_f1_m: 0.8548 - val_recall_m: 0.7789 - val_precision_m: 0.9527
Epoch 34/50
285/285 [==============================] - 7s 24ms/step - loss: 0.4379 - accuracy: 0.7760 - f1_m: 0.8631 - recall_m: 0.7732 - precision_m: 0.9892 - val_loss: 0.5063 - val_accuracy: 0.7713 - val_f1_m: 0.8693 - val_recall_m: 0.8035 - val_precision_m: 0.9519
Epoch 35/50
285/285 [==============================] - 7s 24ms/step - loss: 0.4252 - accuracy: 0.7784 - f1_m: 0.8651 - recall_m: 0.7740 - precision_m: 0.9909 - val_loss: 0.5648 - val_accuracy: 0.7

In [23]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.5044648412748927, 0.5194099378881988, 0.4567736645605075, None)

In [24]:
# deception_pred_test

In [25]:
# unique, counts = np.unique(deception_pred_test_round, return_counts=True)

In [26]:
# print(unique, counts)

In [27]:
# deception_model.history.history['val_f1_m']

### One hot encodings

In [28]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [29]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [30]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = models_nn.create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df_full,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6943 - acc: 0.1703 - f1_m: 0.2279 - precision_m: 0.9309 - recall_m: 0.1330 - val_loss: 0.7145 - val_acc: 0.2854 - val_

143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.3187 - f1_m: 0.4467 - precision_m: 0.9493 - recall_m: 0.2961 - val_loss: 0.6949 - val_acc: 0.4771 - val_f1_m: 0.6288 - val_precision_m: 0.9542 - val_recall_m: 0.4718
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.6450 - f1_m: 0.7714 - precision_m: 0.9541 - recall_m: 0.6597 - val_loss: 0.6935 - val_acc: 0.6522 - val_f1_m: 0.7823 - val_precision_m: 0.9538 - val_recall_m: 0.6658
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.6444 - f1_m: 0.7716 - precision_m: 0.9539 - recall_m: 0.6600 - val_loss: 0.6929 - val_acc: 0.6763 - val_f1_m: 0.8010 - val_precision_m: 0.9523 - val_recall_m: 0.6935
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6930 - acc: 0.6987 - f1_m: 0.8181 - precision_m: 0.9527 - recall_m: 0.7198 - val_loss: 0.6912 - val_acc: 0.6979 - val_f1_m: 0.8168 - val_precision_m: 0.9527 - val_r

In [31]:
joint_predict = joint_full_model.predict(pred_test_df_full)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5034976515767985, 0.5151769195189639, 0.4556258007998503, None)

In [32]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 645 1629]


In [33]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = models_nn.create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6854 - acc: 0.7355 - f1_m: 0.8201 - precision_m: 0.8467 - recall_m: 0.8099 - val_loss: 0.7123 - val_acc: 0.7234 - val_f1

143/143 [==============================] - 0s 3ms/step - loss: 0.6168 - acc: 0.7202 - f1_m: 0.8187 - precision_m: 0.9215 - recall_m: 0.7391 - val_loss: 0.7702 - val_acc: 0.5686 - val_f1_m: 0.7051 - val_precision_m: 0.8686 - val_recall_m: 0.5958
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6132 - acc: 0.7239 - f1_m: 0.8181 - precision_m: 0.9224 - recall_m: 0.7388 - val_loss: 0.7710 - val_acc: 0.5660 - val_f1_m: 0.7025 - val_precision_m: 0.8692 - val_recall_m: 0.5920
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6102 - acc: 0.7215 - f1_m: 0.8178 - precision_m: 0.9235 - recall_m: 0.7368 - val_loss: 0.7686 - val_acc: 0.5664 - val_f1_m: 0.7032 - val_precision_m: 0.8693 - val_recall_m: 0.5930
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6079 - acc: 0.7264 - f1_m: 0.8232 - precision_m: 0.9241 - recall_m: 0.7447 - val_loss: 0.7826 - val_acc: 0.5541 - val_f1_m: 0.6901 - val_precision_m: 0.8672 - val_r

In [34]:
joint_predict = joint_full_model.predict(pred_test_df)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.49588409980294745, 0.4910575380793304, 0.44122510778870316, None)

In [35]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 959 1315]


## Weighted against Throughput, WorkTime, PC Agreement & Text Length

In [36]:
# Train weighted encodings
pred_df_full_throughput, pred_df_throughput, pred_df_full_worktime, pred_df_worktime, pred_df_full_agreement, pred_df_agreement, pred_df_full_textlength, pred_df_textlength, pred_df_full_special, pred_df_special = metadata_options.construct_weighted_dataframe(indices_train, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_df, pred_df_full)

In [37]:
# Test weighted encodings
pred_df_full_throughput_test, pred_df_throughput_test, pred_df_full_worktime_test, pred_df_worktime_test, pred_df_full_agreement_test, pred_df_agreement_test, pred_df_full_textlength_test, pred_df_textlength_test, pred_df_full_special_test, pred_df_special_test = metadata_options.construct_weighted_dataframe(indices_test, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_test_df, pred_test_df_full)

## Throughput only

### Joint Model with one hot encoding 

In [38]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_throughput_test,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6933 - acc: 0.7201 - f1_m: 0.8177 - precision_m: 0.9488 - recall_m: 0.7484 - val_loss: 0.6900 

143/143 [==============================] - 0s 3ms/step - loss: 0.6926 - acc: 0.5080 - f1_m: 0.6533 - precision_m: 0.9499 - recall_m: 0.5031 - val_loss: 0.6877 - val_acc: 0.6284 - val_f1_m: 0.7653 - val_precision_m: 0.9506 - val_recall_m: 0.6427
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6924 - acc: 0.5629 - f1_m: 0.7081 - precision_m: 0.9546 - recall_m: 0.5674 - val_loss: 0.6869 - val_acc: 0.6359 - val_f1_m: 0.7707 - val_precision_m: 0.9503 - val_recall_m: 0.6503
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6925 - acc: 0.6062 - f1_m: 0.7370 - precision_m: 0.9544 - recall_m: 0.6134 - val_loss: 0.6883 - val_acc: 0.5923 - val_f1_m: 0.7362 - val_precision_m: 0.9490 - val_recall_m: 0.6039
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6924 - acc: 0.5263 - f1_m: 0.6756 - precision_m: 0.9532 - recall_m: 0.5278 - val_loss: 0.6868 - val_acc: 0.6315 - val_f1_m: 0.7674 - val_precision_m: 0.9501 - val_r

In [39]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.49942379937049447, 0.4971876238932206, 0.42721298309062583, None)

In [40]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 804 1470]


In [41]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [42]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_throughput_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6943 - acc: 0.6094 - f1_m: 0.7196 - precision_m: 0.8512 - recall_m: 0.6513 - val_loss: 0.6911 - 

143/143 [==============================] - 0s 3ms/step - loss: 0.6608 - acc: 0.7180 - f1_m: 0.8181 - precision_m: 0.9126 - recall_m: 0.7437 - val_loss: 0.6749 - val_acc: 0.6113 - val_f1_m: 0.7405 - val_precision_m: 0.8829 - val_recall_m: 0.6400
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6597 - acc: 0.7038 - f1_m: 0.8065 - precision_m: 0.9133 - recall_m: 0.7250 - val_loss: 0.6758 - val_acc: 0.6091 - val_f1_m: 0.7376 - val_precision_m: 0.8844 - val_recall_m: 0.6349
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6585 - acc: 0.6996 - f1_m: 0.8047 - precision_m: 0.9124 - recall_m: 0.7227 - val_loss: 0.6625 - val_acc: 0.6442 - val_f1_m: 0.7682 - val_precision_m: 0.8857 - val_recall_m: 0.6806
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6572 - acc: 0.7006 - f1_m: 0.8050 - precision_m: 0.9129 - recall_m: 0.7224 - val_loss: 0.6654 - val_acc: 0.6372 - val_f1_m: 0.7624 - val_precision_m: 0.8854 - val_r

In [43]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5194269825918763, 0.5387607182913445, 0.49358289997044175, None)

In [44]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 770 1504]


## WorkTime only

### Joint Model with one hot encoding 

In [45]:
print('Joint full model with one hot encoding, predicting deception, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_worktime_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by worktime
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6935 - acc: 0.6026 - f1_m: 0.6912 - precision_m: 0.8533 - recall_m: 0.6161 - val_loss: 0.6884 - 

143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.7161 - f1_m: 0.8019 - precision_m: 0.9524 - recall_m: 0.7400 - val_loss: 0.6926 - val_acc: 0.4617 - val_f1_m: 0.6130 - val_precision_m: 0.9553 - val_recall_m: 0.4540
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6930 - acc: 0.4087 - f1_m: 0.5480 - precision_m: 0.9523 - recall_m: 0.4043 - val_loss: 0.6910 - val_acc: 0.5528 - val_f1_m: 0.6999 - val_precision_m: 0.9524 - val_recall_m: 0.5552
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6930 - acc: 0.8263 - f1_m: 0.8988 - precision_m: 0.9520 - recall_m: 0.8627 - val_loss: 0.6911 - val_acc: 0.5765 - val_f1_m: 0.7209 - val_precision_m: 0.9540 - val_recall_m: 0.5809
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.7773 - f1_m: 0.8661 - precision_m: 0.9523 - recall_m: 0.8082 - val_loss: 0.6870 - val_acc: 0.8197 - val_f1_m: 0.8994 - val_precision_m: 0.9527 - val_r

In [46]:
joint_predict = joint_full_model.predict(pred_df_full_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5058254682380556, 0.5157426985595348, 0.4944200422970555, None)

In [47]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 338 1936]


In [48]:
print('Joint full model with one hot encoding, predicting rapport, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_worktime_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by worktime
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6915 - acc: 0.7667 - f1_m: 0.8571 - precision_m: 0.8758 - recall_m: 0.8454 - val_loss: 0.6848 - va

143/143 [==============================] - 0s 3ms/step - loss: 0.6549 - acc: 0.6436 - f1_m: 0.7590 - precision_m: 0.9096 - recall_m: 0.6537 - val_loss: 0.7045 - val_acc: 0.5695 - val_f1_m: 0.6959 - val_precision_m: 0.8871 - val_recall_m: 0.5757
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6545 - acc: 0.6496 - f1_m: 0.7610 - precision_m: 0.9101 - recall_m: 0.6572 - val_loss: 0.6823 - val_acc: 0.5906 - val_f1_m: 0.7182 - val_precision_m: 0.8848 - val_recall_m: 0.6069
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6546 - acc: 0.6508 - f1_m: 0.7622 - precision_m: 0.9074 - recall_m: 0.6606 - val_loss: 0.7057 - val_acc: 0.5739 - val_f1_m: 0.7012 - val_precision_m: 0.8879 - val_recall_m: 0.5823
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6545 - acc: 0.6471 - f1_m: 0.7616 - precision_m: 0.9090 - recall_m: 0.6585 - val_loss: 0.6855 - val_acc: 0.5901 - val_f1_m: 0.7178 - val_precision_m: 0.8848 - val_r

In [49]:
joint_predict = joint_full_model.predict(pred_df_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5183826101641758, 0.5393818063411281, 0.4747174525121941, None)

In [50]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 915 1359]


## PC Agreement only

### Joint Model with one hot encoding 

In [51]:
print('Joint full model with one hot encoding, predicting deception, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_full_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_agreement, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_agreement_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by PC Agreement
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6934 - acc: 0.1202 - f1_m: 0.1379 - precision_m: 0.7544 - recall_m: 0.0818 - val_loss: 0.705

143/143 [==============================] - 0s 3ms/step - loss: 0.6919 - acc: 0.5959 - f1_m: 0.7322 - precision_m: 0.9452 - recall_m: 0.6020 - val_loss: 0.6866 - val_acc: 0.6412 - val_f1_m: 0.7763 - val_precision_m: 0.9498 - val_recall_m: 0.6580
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6919 - acc: 0.5839 - f1_m: 0.7276 - precision_m: 0.9531 - recall_m: 0.5928 - val_loss: 0.6874 - val_acc: 0.6271 - val_f1_m: 0.7649 - val_precision_m: 0.9491 - val_recall_m: 0.6424
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6919 - acc: 0.4915 - f1_m: 0.6421 - precision_m: 0.9550 - recall_m: 0.4896 - val_loss: 0.6898 - val_acc: 0.5884 - val_f1_m: 0.7323 - val_precision_m: 0.9521 - val_recall_m: 0.5974
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6917 - acc: 0.6047 - f1_m: 0.7440 - precision_m: 0.9526 - recall_m: 0.6139 - val_loss: 0.6871 - val_acc: 0.6346 - val_f1_m: 0.7710 - val_precision_m: 0.9497 - val_r

In [52]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5398435696482861, 0.5877198574167906, 0.4434563954688726, None)

In [53]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1259 1015]


In [54]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [55]:
print('Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_agreement, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_agreement_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6931 - acc: 0.5499 - f1_m: 0.5473 - precision_m: 0.5552 - recall_m: 0.5670 - val_loss: 0.6969 

Epoch 29/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6595 - acc: 0.7123 - f1_m: 0.8185 - precision_m: 0.8922 - recall_m: 0.7587 - val_loss: 0.7158 - val_acc: 0.6451 - val_f1_m: 0.7709 - val_precision_m: 0.8789 - val_recall_m: 0.6887
Epoch 30/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6591 - acc: 0.7090 - f1_m: 0.8132 - precision_m: 0.8910 - recall_m: 0.7517 - val_loss: 0.7069 - val_acc: 0.6495 - val_f1_m: 0.7743 - val_precision_m: 0.8784 - val_recall_m: 0.6948
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6583 - acc: 0.7062 - f1_m: 0.8142 - precision_m: 0.8930 - recall_m: 0.7508 - val_loss: 0.7189 - val_acc: 0.6376 - val_f1_m: 0.7631 - val_precision_m: 0.8829 - val_recall_m: 0.6745
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6570 - acc: 0.7025 - f1_m: 0.8097 - precision_m: 0.8946 - recall_m: 0.7418 - val_loss: 0.7252 - val_acc: 0.6262 - val_f1_m: 0.7529 - val_precision_m: 0.

In [56]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4528216982240904, 0.427387829430787, 0.4361543859649123, None)

In [57]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 505 1769]


## Text Length only

### Joint Model with one hot encoding 

In [58]:
print('Joint full model with one hot encoding, predicting deception, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_full_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_textlength, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_textlength_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by text length
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_27 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6933 - acc: 0.9446 - f1_m: 0.9671 - precision_m: 0.9430 - recall_m: 0.9930 - val_loss: 0.6901

143/143 [==============================] - 0s 3ms/step - loss: 0.6924 - acc: 0.6032 - f1_m: 0.7413 - precision_m: 0.9560 - recall_m: 0.6109 - val_loss: 0.6801 - val_acc: 0.6583 - val_f1_m: 0.7885 - val_precision_m: 0.9526 - val_recall_m: 0.6747
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6923 - acc: 0.6487 - f1_m: 0.7802 - precision_m: 0.9538 - recall_m: 0.6646 - val_loss: 0.6842 - val_acc: 0.6293 - val_f1_m: 0.7660 - val_precision_m: 0.9513 - val_recall_m: 0.6431
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6924 - acc: 0.4409 - f1_m: 0.5893 - precision_m: 0.9531 - recall_m: 0.4364 - val_loss: 0.6865 - val_acc: 0.6069 - val_f1_m: 0.7471 - val_precision_m: 0.9518 - val_recall_m: 0.6169
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6923 - acc: 0.6501 - f1_m: 0.7782 - precision_m: 0.9503 - recall_m: 0.6624 - val_loss: 0.6930 - val_acc: 0.5409 - val_f1_m: 0.6897 - val_precision_m: 0.9547 - val_r

In [59]:
joint_predict = joint_full_model.predict(pred_df_full_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.504055695729371, 0.5215078630897317, 0.39455139714043225, None)

In [60]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1044 1230]


In [61]:
print('Joint full model with one hot encoding, predicting rapport, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_textlength, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_textlength_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by text length
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6910 - acc: 0.2025 - f1_m: 0.1207 - precision_m: 0.2772 - recall_m: 0.0803 - val_loss: 0.6971 -

143/143 [==============================] - 0s 3ms/step - loss: 0.6522 - acc: 0.7292 - f1_m: 0.8274 - precision_m: 0.9052 - recall_m: 0.7652 - val_loss: 0.6786 - val_acc: 0.6838 - val_f1_m: 0.8015 - val_precision_m: 0.8726 - val_recall_m: 0.7438
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6516 - acc: 0.7306 - f1_m: 0.8282 - precision_m: 0.9055 - recall_m: 0.7670 - val_loss: 0.6790 - val_acc: 0.6860 - val_f1_m: 0.8032 - val_precision_m: 0.8726 - val_recall_m: 0.7468
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6511 - acc: 0.7489 - f1_m: 0.8427 - precision_m: 0.9036 - recall_m: 0.7929 - val_loss: 0.7066 - val_acc: 0.6319 - val_f1_m: 0.7596 - val_precision_m: 0.8717 - val_recall_m: 0.6765
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6506 - acc: 0.7279 - f1_m: 0.8269 - precision_m: 0.9047 - recall_m: 0.7647 - val_loss: 0.6999 - val_acc: 0.6407 - val_f1_m: 0.7670 - val_precision_m: 0.8716 - val_r

In [62]:
joint_predict = joint_full_model.predict(pred_df_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5023108518321369, 0.504427728954074, 0.47823456805725817, None)

In [63]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 712 1562]


## Other special options (either SP1, SP2, SP3, RAND_UNI, or RAND_NORM)

### Joint Model with one hot encoding 

In [64]:
print('Joint full model with one hot encoding, predicting deception, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_full_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_special, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_special_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by special option
Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6933 - acc: 0.4081 - f1_m: 0.5314 - precision_m: 0.9437 - recall_m: 0.3963 - val_loss: 0.6

143/143 [==============================] - 0s 3ms/step - loss: 0.6926 - acc: 0.6071 - f1_m: 0.7476 - precision_m: 0.9540 - recall_m: 0.6176 - val_loss: 0.6832 - val_acc: 0.6025 - val_f1_m: 0.7434 - val_precision_m: 0.9497 - val_recall_m: 0.6132
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6926 - acc: 0.5714 - f1_m: 0.7169 - precision_m: 0.9546 - recall_m: 0.5790 - val_loss: 0.6896 - val_acc: 0.5295 - val_f1_m: 0.6792 - val_precision_m: 0.9525 - val_recall_m: 0.5299
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6925 - acc: 0.5470 - f1_m: 0.6936 - precision_m: 0.9538 - recall_m: 0.5509 - val_loss: 0.6845 - val_acc: 0.5844 - val_f1_m: 0.7289 - val_precision_m: 0.9487 - val_recall_m: 0.5941
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6924 - acc: 0.4513 - f1_m: 0.5994 - precision_m: 0.9573 - recall_m: 0.4454 - val_loss: 0.6916 - val_acc: 0.5088 - val_f1_m: 0.6596 - val_precision_m: 0.9518 - val_r

In [65]:
joint_predict = joint_full_model.predict(pred_df_full_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5024525865503867, 0.5130913836394873, 0.37856311905259943, None)

In [66]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1121 1153]


In [67]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [68]:
print('Joint full model with one hot encoding, predicting rapport, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_special, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_special_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by special option
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6918 - acc: 0.3639 - f1_m: 0.4371 - precision_m: 0.8152 - recall_m: 0.3154 - val_loss: 0.697

143/143 [==============================] - 0s 3ms/step - loss: 0.6812 - acc: 0.7496 - f1_m: 0.8492 - precision_m: 0.8757 - recall_m: 0.8265 - val_loss: 0.6861 - val_acc: 0.7397 - val_f1_m: 0.8411 - val_precision_m: 0.8857 - val_recall_m: 0.8032
Epoch 30/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6812 - acc: 0.7397 - f1_m: 0.8417 - precision_m: 0.8760 - recall_m: 0.8126 - val_loss: 0.6743 - val_acc: 0.7731 - val_f1_m: 0.8663 - val_precision_m: 0.8858 - val_recall_m: 0.8498
Epoch 31/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6810 - acc: 0.7623 - f1_m: 0.8583 - precision_m: 0.8762 - recall_m: 0.8435 - val_loss: 0.6798 - val_acc: 0.7529 - val_f1_m: 0.8517 - val_precision_m: 0.8841 - val_recall_m: 0.8238
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6807 - acc: 0.7557 - f1_m: 0.8533 - precision_m: 0.8743 - recall_m: 0.8356 - val_loss: 0.6771 - val_acc: 0.7643 - val_f1_m: 0.8605 - val_precision_m: 0.8851 - val_r

In [69]:
joint_predict = joint_full_model.predict(pred_df_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5394387045086416, 0.5506320281582887, 0.5421281583158654, None)

In [70]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 397 1877]
